<div class="alert alert-block alert-info">
<b>Number of points for this notebook:</b> 2
<br>
<b>Deadline:</b> March 17, 2021 (Wednesday) 23:00
</div>

# Exercise 3.2. Convolutional networks. VGG-style network.

In the second part you need to train a convolutional neural network with an architecture inspired by a VGG-network [(Simonyan \& Zisserman, 2015)](https://arxiv.org/abs/1409.1556).

In [92]:
skip_training = True  # Set this flag to True before validation and submission

In [2]:
# During evaluation, this cell sets skip_training to True
# skip_training = True

In [3]:
import os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torchvision
import torchvision.transforms as transforms

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import tools
import tests

In [4]:
# When running on your own computer, you can specify the data directory by:
# data_dir = tools.select_data_dir('/your/local/data/directory')
data_dir = tools.select_data_dir()

The data directory is /coursedata


In [5]:
# Select the device for training (use GPU if you have one)
#device = torch.device('cuda:0')
device = torch.device('cpu')

In [6]:
if skip_training:
    # The models are always evaluated on CPU
    device = torch.device("cpu")

## FashionMNIST dataset

Let us use the FashionMNIST dataset. It consists of 60,000 training images of 10 classes: 'T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot'.

In [7]:
transform = transforms.Compose([
    transforms.ToTensor(),  # Transform to tensor
    transforms.Normalize((0.5,), (0.5,))  # Scale images to [-1, 1]
])

trainset = torchvision.datasets.FashionMNIST(root=data_dir, train=True, download=True, transform=transform)
testset = torchvision.datasets.FashionMNIST(root=data_dir, train=False, download=True, transform=transform)

classes = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal',
           'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

trainloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=5, shuffle=False)

# VGG-style network

Let us now define a convolution neural network with an architecture inspired by the [VGG-net](https://arxiv.org/abs/1409.1556).

The architecture:
- A block of three convolutional layers with:
    - 3x3 kernel
    - 20 output channels
    - one pixel zero-pading on both sides
    - 2d batch normalization after each convolutional layer
    - ReLU nonlinearity after each 2d batch normalization layer
- Max pooling layer with 2x2 kernel and stride 2.
- A block of three convolutional layers with:
    - 3x3 kernel
    - 40 output channels
    - one pixel zero-pading on both sides
    - 2d batch normalization after each convolutional layer
    - ReLU nonlinearity after each 2d batch normalization layer
- Max pooling layer with 2x2 kernel and stride 2.
- One convolutional layer with:
    - 3x3 kernel
    - 60 output channels
    - *no padding*
    - 2d batch normalization after the convolutional layer
    - ReLU nonlinearity after the 2d batch normalization layer
- One convolutional layer with:
    - 1x1 kernel
    - 40 output channels
    - *no padding*
    - 2d batch normalization after the convolutional layer
    - ReLU nonlinearity after the 2d batch normalization layer
- One convolutional layer with:
    - 1x1 kernel
    - 20 output channels
    - *no padding*
    - 2d batch normalization after the convolutional layer
    - ReLU nonlinearity after the 2d batch normalization layer
- Global average pooling (compute the average value of each channel across all the input locations):
    - 5x5 kernel (the input of the layer should be 5x5)
- A fully-connected layer with 10 outputs (no nonlinearity)

Notes:
* Batch normalization is expected to be right after a convolutional layer, before nonlinearity.
* We recommend that you check the number of modules with trainable parameters in your network.

In [75]:
class VGGNet(nn.Module):
    def __init__(self):
        super(VGGNet, self).__init__()
        # YOUR CODE HERE
        self.block1 = nn.Sequential(
            nn.Conv2d(1, 20, 3, padding = 1),
            nn.BatchNorm2d(20),
            nn.ReLU(),
            nn.Conv2d(20, 20, 3, padding = 1),
            nn.BatchNorm2d(20),
            nn.ReLU(),
            nn.Conv2d(20, 20, 3, padding = 1),
            nn.BatchNorm2d(20),
            nn.ReLU(),
            nn.MaxPool2d(2,2)
        )
        
        self.block2 = nn.Sequential(
            nn.Conv2d(20, 40, 3, padding = 1),
            nn.BatchNorm2d(40),
            nn.ReLU(),
            nn.Conv2d(40, 40, 3, padding = 1),
            nn.BatchNorm2d(40),
            nn.ReLU(),
            nn.Conv2d(40, 40, 3, padding = 1),
            nn.BatchNorm2d(40),
            nn.ReLU(),
            nn.MaxPool2d(2,2)
        )
        
        self.block3 = nn.Sequential(
            nn.Conv2d(40, 60, 3),
            nn.BatchNorm2d(60),
            nn.ReLU(),
            nn.Conv2d(60, 40, 1),
            nn.BatchNorm2d(40),
            nn.ReLU(),
            nn.Conv2d(40, 20, 1),
            nn.BatchNorm2d(20),
            nn.ReLU(),
        )
        self.block4 = nn.Sequential(
            nn.AvgPool2d(5) #find right, 
        )
        self.block5 = nn.Sequential(
            nn.Linear(20,10) #find input size
        )

    def forward(self, x, verbose=False):
        """
        Args:
          x of shape (batch_size, 1, 28, 28): Input images.
          verbose: True if you want to print the shapes of the intermediate variables.
        
        Returns:
          y of shape (batch_size, 10): Outputs of the network.
        """
        # YOUR CODE HERE
        if verbose == True:
            y = self.block1(x)
            print("Shape of Variable after Block1:  ", y.shape)
            y = self.block2(y)
            print("Shape of Variable after Block2:  ", y.shape)
            y = self.block3(y)
            print("Shape of Variable after Block3:  ", y.shape)
            y = self.block4(y)
            print("Shape of Variable after Block4:  ", y.shape)
            y = y.flatten(1)
            print("Shape of Variable after flatten:  ", y.shape)
            y = self.block5(y)
            
            print("Shape of Variable after Block5:  ", y.shape)
        else:
            y = self.block1(x)
            y = self.block2(y)
            y = self.block3(y)
            y = self.block4(y)
            y = y.flatten(1)
            y = self.block5(y)
        return y 

In [76]:
def test_VGGNet_shapes():
    net = VGGNet()
    net.to(device)

    # Feed a batch of images from the training data to test the network
    with torch.no_grad():
        images, labels = iter(trainloader).next()
        images = images.to(device)
        print('Shape of the input tensor:', images.shape)

        y = net(images, verbose=True)
        assert y.shape == torch.Size([trainloader.batch_size, 10]), f"Bad y.shape: {y.shape}"

    print('Success')

test_VGGNet_shapes()

Shape of the input tensor: torch.Size([32, 1, 28, 28])
Shape of Variable after Block1:   torch.Size([32, 20, 14, 14])
Shape of Variable after Block2:   torch.Size([32, 40, 7, 7])
Shape of Variable after Block3:   torch.Size([32, 20, 5, 5])
Shape of Variable after Block4:   torch.Size([32, 20, 1, 1])
Shape of Variable after flatten:   torch.Size([32, 20])
Shape of Variable after Block5:   torch.Size([32, 10])
Success


In [77]:
tests.test_vgg_net(VGGNet)

y: tensor([[ 10.0382,  10.0382,  10.0382,  10.0382,  10.0382, -10.0382, -10.0382,
         -10.0382, -10.0382, -10.0382]], grad_fn=<AddmmBackward>)
expected: tensor([ 10.0382,  10.0382,  10.0382,  10.0382,  10.0382, -10.0382, -10.0382,
        -10.0382, -10.0382, -10.0382])
Success


# Train the network

In [78]:
# This function computes the accuracy on the test dataset
def compute_accuracy(net, testloader):
    net.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(device), labels.to(device)
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return correct / total

### Training loop

Your task is to implement the training loop. The recommended hyperparameters:
* Adam optimizer with learning rate 0.01.
* Cross-entropy loss. Note that we did not use softmax nonlinearity in the final layer of our network. Therefore, we need to use a loss function with log_softmax implemented, such as [`nn.CrossEntropyLoss`](https://pytorch.org/docs/stable/nn.html#torch.nn.CrossEntropyLoss).
* Number of epochs: 10

We recommend you to use function `compute_accuracy()` defined above to track the accaracy during training. The test accuracy should be above 0.89.

**Note: function `compute_accuracy()` sets the network into the evaluation mode which changes the way the batch statistics are computed in batch normalization. You need to set the network into the training mode (by calling `net.train()`) when you want to perform training.**

In [79]:
net = VGGNet()

In [88]:
# Implement the training loop in this cell
if not skip_training:
    # YOUR CODE HERE
    optimizer = optim.Adam(net.parameters(), lr=0.01)
    criterion = nn.CrossEntropyLoss()
    epochs = 10
    net.train()
    for epoch in range(epochs):
        print("Epoch number:  ", epoch)
        for i, data in enumerate(trainloader, 0):
            images, labels = data
            optimizer.zero_grad()
            output = net(images)
            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()
            if i % 32 == 31:
                acc = compute_accuracy(net, testloader)
                print("Accuracy:  ", acc)
                net.train()

Epoch number:   0
Accuracy:   0.7429
Accuracy:   0.7529
Accuracy:   0.7169
Accuracy:   0.7724
Accuracy:   0.7839
Accuracy:   0.7812
Accuracy:   0.7149
Accuracy:   0.7595
Accuracy:   0.7893
Accuracy:   0.8034
Accuracy:   0.795
Accuracy:   0.7975
Accuracy:   0.8254
Accuracy:   0.8169
Accuracy:   0.8088
Accuracy:   0.8003
Accuracy:   0.8127
Accuracy:   0.8118
Accuracy:   0.8209
Accuracy:   0.8241
Accuracy:   0.816
Accuracy:   0.7836
Accuracy:   0.813
Accuracy:   0.811
Accuracy:   0.826
Accuracy:   0.3152
Accuracy:   0.8374
Accuracy:   0.8181
Accuracy:   0.8465
Accuracy:   0.8413
Accuracy:   0.8534
Accuracy:   0.8419
Accuracy:   0.7817
Accuracy:   0.8218
Accuracy:   0.8232
Accuracy:   0.8543
Accuracy:   0.8491
Accuracy:   0.8503
Accuracy:   0.8328
Accuracy:   0.8321
Accuracy:   0.8313
Accuracy:   0.8334
Accuracy:   0.8454
Accuracy:   0.8405
Accuracy:   0.8539
Accuracy:   0.8488
Accuracy:   0.8681
Accuracy:   0.8572
Accuracy:   0.8517
Accuracy:   0.8614
Accuracy:   0.8548
Accuracy:   0.8532

Accuracy:   0.9207
Accuracy:   0.9199
Accuracy:   0.9087
Accuracy:   0.9207
Accuracy:   0.9234
Accuracy:   0.9097
Accuracy:   0.9194
Accuracy:   0.9217
Accuracy:   0.921
Accuracy:   0.9199
Accuracy:   0.9162
Accuracy:   0.9121
Accuracy:   0.9171
Accuracy:   0.9124
Accuracy:   0.9198
Accuracy:   0.9223
Accuracy:   0.9167
Accuracy:   0.9205
Accuracy:   0.9127
Accuracy:   0.9106
Accuracy:   0.9178
Accuracy:   0.913
Accuracy:   0.9141
Accuracy:   0.9217
Accuracy:   0.9242
Accuracy:   0.9189
Accuracy:   0.9153
Accuracy:   0.9245
Accuracy:   0.9112
Accuracy:   0.9186
Accuracy:   0.9213
Accuracy:   0.9189
Accuracy:   0.9228
Accuracy:   0.918
Accuracy:   0.9137
Accuracy:   0.9131
Accuracy:   0.9229
Accuracy:   0.9195
Epoch number:   8
Accuracy:   0.9203
Accuracy:   0.9142
Accuracy:   0.9196
Accuracy:   0.926
Accuracy:   0.9207
Accuracy:   0.9201
Accuracy:   0.9226
Accuracy:   0.912
Accuracy:   0.9213
Accuracy:   0.9237
Accuracy:   0.9109
Accuracy:   0.9184
Accuracy:   0.918
Accuracy:   0.9216


In [89]:
# Save the model to disk (the pth-files will be submitted automatically together with your notebook)
# Set confirm=False if you do not want to be asked for confirmation before saving.
if not skip_training:
    tools.save_model(net, '2_vgg_net.pth', confirm=True)

Do you want to save the model (type yes to confirm)? yes
Model saved to 2_vgg_net.pth.


In [90]:
if skip_training:
    net = VGGNet()
    tools.load_model(net, '2_vgg_net.pth', device)

In [91]:
# Compute the accuracy on the test set
accuracy = compute_accuracy(net, testloader)
print(f'Accuracy of the VGG net on the test images: {accuracy: .3f}')
assert accuracy > 0.89, 'Poor accuracy'
print('Success')

Accuracy of the VGG net on the test images:  0.915
Success
